In [1]:
%run ../MLHCExperiment-Common-Val17Test18.ipynb

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
all_visits = read_data()
all_visits = all_visits.where(all_visits["AdjCount"] > 0).dropna()

In [3]:
epochs = 1
patience = 5
trials = 1

In [6]:
def run_one_step_ahead_rnn(filename, name, features, label):
    print(name)
    X = all_visits[features]
    y = all_visits[label]
    
    mses = list()

    OSH_DATES = X.loc[VAL_END:TEST_END].index

    start = time.time()
    predictions = list()

    features_min_max = preprocessing.MinMaxScaler()
    X_train, y_train, X_val, y_val, X_test, y_test = split_for_baseline_and_nn(X, y)

    X_train_t = reshape_for_rnn(features_min_max.fit_transform(X_train))
    X_val_t = reshape_for_rnn(features_min_max.transform(X_val))
    model = compile_rnn_weights_from_file(filename, X_train, 32, 32)

    # fit time series model
    for i in range(len(OSH_DATES) - 1):
        start2 = time.time()
        prev_date = OSH_DATES[i]
        next_date = OSH_DATES[i + 1]
        if i == 0:
            model.fit(X_val_t, y_val.values, epochs=1, shuffle=False, verbose=0)
        if i > 0:
            X_train_t = reshape_for_rnn(features_min_max.transform(X_test.loc[prev_date].values.reshape(1, -1)))
            y_train = pd.Series(y_test.loc[prev_date])
            history = model.fit(X_train_t, y_train.values, epochs=1, shuffle=False, verbose=0)

        X_test_t = reshape_for_rnn(features_min_max.transform(X_test.loc[next_date].values.reshape(1, -1)))
        y_pred = model.predict(X_test_t, batch_size=1)
        predictions.extend(y_pred)
        end2 = time.time()
        if i % 14 == 0:
            print("14d ending with %s done; last one took %.2f s" % (str(next_date), end2-start2))

    mses.append(mean_squared_error(predictions, y_test))
    end = time.time()
    print("Trial completed in %.2f s" % (end - start))
    print("Average MSE so far: %.3f (%.3f)" % (np.mean(mses), np.std(mses)))
    return mses, y_pred

In [7]:
filenames = prebuilt_rnn_models["32-32"]

mses = list()

for filename in filenames:
    mse, _ = run_one_step_ahead_rnn("../" + filename, filename, BASE_FEATURES, ALL_VISITS_LABEL)
    mses.append(mse)

print_mse_metrics(mses)

RNNModels/32-32Best/Dynamic-RNN-AllVisits-32-32-Trial0-1562316203.5722883.h5
14d ending with 2018-01-01 00:00:00 done; last one took 7.98 s
14d ending with 2018-01-15 00:00:00 done; last one took 0.02 s
14d ending with 2018-01-29 00:00:00 done; last one took 0.02 s
14d ending with 2018-02-12 00:00:00 done; last one took 0.02 s
14d ending with 2018-02-26 00:00:00 done; last one took 0.02 s
14d ending with 2018-03-13 00:00:00 done; last one took 0.02 s
14d ending with 2018-03-27 00:00:00 done; last one took 0.02 s
14d ending with 2018-04-10 00:00:00 done; last one took 0.02 s
14d ending with 2018-04-24 00:00:00 done; last one took 0.02 s
14d ending with 2018-05-08 00:00:00 done; last one took 0.02 s
14d ending with 2018-05-22 00:00:00 done; last one took 0.02 s
14d ending with 2018-06-05 00:00:00 done; last one took 0.02 s
14d ending with 2018-06-19 00:00:00 done; last one took 0.02 s
14d ending with 2018-07-03 00:00:00 done; last one took 0.02 s
14d ending with 2018-07-17 00:00:00 done;

In [8]:
filenames_r = prebuilt_rnn_models["32-32R"]

mses_r = list()

for filename in filenames_r:
    mse, _ = run_one_step_ahead_rnn("../" + filename, filename, BASE_FEATURES, ICD8_LABEL)
    mses_r.append(mse)

print_mse_metrics(mses_r)

RNNModels/32-32Resp/Dynamic-RNN-RespVisits-32-32-Trial4-1562366029.9926655.h5
14d ending with 2018-01-01 00:00:00 done; last one took 15.88 s
14d ending with 2018-01-15 00:00:00 done; last one took 0.02 s
14d ending with 2018-01-29 00:00:00 done; last one took 0.02 s
14d ending with 2018-02-12 00:00:00 done; last one took 0.02 s
14d ending with 2018-02-26 00:00:00 done; last one took 0.02 s
14d ending with 2018-03-13 00:00:00 done; last one took 0.02 s
14d ending with 2018-03-27 00:00:00 done; last one took 0.03 s
14d ending with 2018-04-10 00:00:00 done; last one took 0.02 s
14d ending with 2018-04-24 00:00:00 done; last one took 0.02 s
14d ending with 2018-05-08 00:00:00 done; last one took 0.02 s
14d ending with 2018-05-22 00:00:00 done; last one took 0.02 s
14d ending with 2018-06-05 00:00:00 done; last one took 0.03 s
14d ending with 2018-06-19 00:00:00 done; last one took 0.03 s
14d ending with 2018-07-03 00:00:00 done; last one took 0.02 s
14d ending with 2018-07-17 00:00:00 don

In [9]:
filenames_nr = prebuilt_rnn_models["32-32NR"]

mses_nr = list()

for filename in filenames_nr:
    mse, _ = run_one_step_ahead_rnn("../" + filename, filename, BASE_FEATURES, ICD8OPP_LABEL)
    mses_nr.append(mse)

print_mse_metrics(mses_nr)

RNNModels/32-32NonResp/Dynamic-RNN-NonRespVisits-32-32-Trial3-1562367965.7343554.h5
14d ending with 2018-01-01 00:00:00 done; last one took 21.02 s
14d ending with 2018-01-15 00:00:00 done; last one took 0.03 s
14d ending with 2018-01-29 00:00:00 done; last one took 0.02 s
14d ending with 2018-02-12 00:00:00 done; last one took 0.02 s
14d ending with 2018-02-26 00:00:00 done; last one took 0.02 s
14d ending with 2018-03-13 00:00:00 done; last one took 0.02 s
14d ending with 2018-03-27 00:00:00 done; last one took 0.02 s
14d ending with 2018-04-10 00:00:00 done; last one took 0.02 s
14d ending with 2018-04-24 00:00:00 done; last one took 0.05 s
14d ending with 2018-05-08 00:00:00 done; last one took 0.02 s
14d ending with 2018-05-22 00:00:00 done; last one took 0.03 s
14d ending with 2018-06-05 00:00:00 done; last one took 0.03 s
14d ending with 2018-06-19 00:00:00 done; last one took 0.03 s
14d ending with 2018-07-03 00:00:00 done; last one took 0.04 s
14d ending with 2018-07-17 00:00: